In [14]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, SpatialDropout1D, LSTM
from keras.models import load_model
from keras.utils.np_utils import to_categorical
import re

In [2]:
data = pd.read_csv('data/RNN - LSTM/LSTMData.txt')
data.columns = ['cumle', 'sinif']
data = data[['cumle', 'sinif']]
print(data[0:5])

                                               cumle sinif
0  koku çok kız çünkü ayva ben tek arkadaş  insan...   kiz
1                 hayır çok sev dizi yani aşırı kız    kiz
2  buna kız de evet ona kız ney kız kızneg sen ma...   kiz
3                                   kız ve çok üzül    kiz
4        kız bu geç ama aynı zaman kır bu nasıl geç    kiz


In [6]:
# Veri setimizin işlenebilmesi için text verileri numaralara çevirmemiz gerekir
# Keras bu işlem için hazır bir mekanizma sunmaktadır
# Tokenizer sınıfı data içerisinde verilen cümleleri analiz ederek kelimelerin sıklıklarını hesaplar
# Parameter: num_words = En sık geçen 25000 kelimeye odaklan, diğerleri önemli değil demek
tokenizer = Tokenizer(split = ' ', num_words = 25000)
# Her bir kelimenin sıklığını(frekansını) hesaplar
tokenizer.fit_on_texts(data['cumle'].values)
# Tüm cümleler tam sayı dizisine dönüştürülür
X = tokenizer.texts_to_sequences(data['cumle'].values)
print(X[0:2])
# Bütün metinlerimiz 400 sütundan oluşan bir dizi ile temsil edilecek
# Çok kısa metinler 0'lar ile doldurulacak, çok uzun metinler ise kesilecek
X = pad_sequences(X,maxlen=400)
print(X[0])

[[1094, 1, 57, 281, 1361, 4, 173, 104, 22, 17, 21, 23, 19, 173, 104, 712, 180], [258, 1, 49, 192, 259, 154, 57]]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0

In [16]:
# Yapay sinir ağı modelimiz
embed_dim = 128
lstm_out = 128
def build_model():
    model = Sequential()
    # Her bir kelimenin temsil edileceği vektör boyutu. Bu örnek için her bir kelime 128 boyutunda
    # bir vektör ile temsil edilir.
    model.add(Embedding(25000, embed_dim, input_length = X.shape[1]))
    model.add(SpatialDropout1D(rate = 0.2))
    model.add(LSTM(lstm_out, dropout = 0.2, recurrent_dropout = 0.2))
    model.add(Dense(4, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

  # Çıktılarımızı kategorik hale getirdik (Opsiyonel)
Y = pd.get_dummies(data['sinif']).values
Y[0]

array([1, 0, 0, 0], dtype=uint8)

In [18]:
# Verinin %90'i train, %10'si test verisi olacak şekilde ayrılacak
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 1)

model = build_model()
# Oluşturulan model train verileri ile eğitilir
# Burada yapay sinir ağını eğitmeye başlıyoruz
# nb_epoch: İterasyon sayısı
model.fit(X_train, Y_train, epochs = 5, batch_size = 32, verbose = 2)
# Train verileri ile model eğitildikten sonra test dataları ile doğruluk oranlarına bakılır.
score = model.evaluate(X_test, Y_test, verbose = 2)
print("score: %.2f" % (score[1]))

Epoch 1/5
 - 39s - loss: 1.1542 - acc: 0.5051
Epoch 2/5
 - 42s - loss: 0.4053 - acc: 0.8872
Epoch 3/5
 - 41s - loss: 0.1368 - acc: 0.9608
Epoch 4/5
 - 40s - loss: 0.0710 - acc: 0.9819
Epoch 5/5
 - 40s - loss: 0.0491 - acc: 0.9888
score: 0.92


In [19]:
# Çıktılarımız => 0: Kızmak, 1: Korkmak, 2: Mutluluk, 3:Üzüntü
outputs = ["Kızmak", "Korkmak", "Mutluluk", "Üzüntü"]

# Verilen örnekler Tokenizer yapısı ile tam sayı dizisine dönüştürülür
# Daha sonra eğitilen modele sırayla verilerek anlam analizi sonuçları elde edilir
# Her Cümlenin yüzde kaç olumlu ve olumsuz olduğuna dair bilgiler çıktı olarak verilir

my_text = ["çok kız ne kadar ayıp şey"]

sequences = tokenizer.texts_to_sequences(my_text)
data = pad_sequences(sequences, maxlen = 400)
predictions = model.predict(data)
print(predictions[0])
predictions = list(predictions)
print(outputs[predictions.index(max(predictions))])

[0.85069966 0.01189544 0.01280428 0.1246006 ]
Kızmak
